Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = "Oscar Fernandez"
COLLABORATORS = "Amrit Chandrasekaran, Tanav Bollam"

## 1. Setup and Database Schema

This section connects to a SQLite database and defines the schema for the core tables used in the project.

In [65]:
import sqlite3
import pandas as pd

# connect to (or create) a SQLite database file in this assignment folder
conn = sqlite3.connect("delivery.db")

# helper function to run SQL and display results as a table
def run_sql(query):
    return pd.read_sql_query(query, conn)


In [66]:
schema_sql = """
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS restaurants;
DROP TABLE IF EXISTS drivers;

CREATE TABLE customers (
    customer_id      INTEGER PRIMARY KEY,
    customer_name    TEXT,
    customer_city    TEXT
);

CREATE TABLE restaurants (
    restaurant_id    INTEGER PRIMARY KEY,
    restaurant_name  TEXT,
    cuisine_type     TEXT
);

CREATE TABLE drivers (
    driver_id     INTEGER PRIMARY KEY,
    driver_name   TEXT,
    driver_rating REAL
);

CREATE TABLE orders (
    order_id                INTEGER PRIMARY KEY,
    customer_id             INTEGER,
    restaurant_id           INTEGER,
    driver_id               INTEGER,
    order_time              TEXT,
    estimated_delivery_time TEXT,
    actual_delivery_time    TEXT,
    delivery_distance       REAL,
    preparation_time        INTEGER,
    weather_conditions      TEXT,
    customer_rating         INTEGER,
    FOREIGN KEY(customer_id)   REFERENCES customers(customer_id),
    FOREIGN KEY(restaurant_id) REFERENCES restaurants(restaurant_id),
    FOREIGN KEY(driver_id)     REFERENCES drivers(driver_id)
);
"""

# it executes all the SQL commands above
conn.executescript(schema_sql)
print("Tables created.")

Tables created.


## 2. Prototype Sample Data

This section uses a small synthetic dataset to prototype the database pipeline and perform SQL analysis of the features before integrating the real-world data.

In [67]:
insert_sql = """
DELETE FROM customers;
DELETE FROM restaurants;
DELETE FROM drivers;

INSERT INTO customers (customer_id, customer_name, customer_city) VALUES
(1, 'Alice Johnson', 'Newark'),
(2, 'Oscar Fernandez', 'Jersey City'),
(3, 'Maria Lopez', 'Hoboken'),
(4, 'John Smith', 'New Brunswick');

INSERT INTO restaurants (restaurant_id, restaurant_name, cuisine_type) VALUES
(1, 'Speedy Pizza', 'Pizza'),
(2, 'Sushi World', 'Japanese'),
(3, 'Taco Fiesta', 'Mexican');

INSERT INTO drivers (driver_id, driver_name, driver_rating) VALUES
(1, 'David Lee', 4.8),
(2, 'Jessica Chen', 4.2),
(3, 'Carlos Diaz', 4.5);
"""
conn.executescript(insert_sql)
print("Replaced sample customers, restaurants, and drivers.")

Replaced sample customers, restaurants, and drivers.


In [68]:
orders_sql = """
DELETE FROM orders;

INSERT INTO orders (
    order_id, customer_id, restaurant_id, driver_id,
    order_time, estimated_delivery_time, actual_delivery_time,
    delivery_distance, preparation_time, weather_conditions,
    customer_rating
) VALUES
-- On-time example
(1001, 1, 1, 1,
 '2025-11-20 18:00:00', '2025-11-20 18:30:00', '2025-11-20 18:28:00',
 3.5, 20, 'Clear', 5),

-- Slightly late
(1002, 2, 2, 2,
 '2025-11-20 19:10:00', '2025-11-20 19:40:00', '2025-11-20 19:50:00',
 5.0, 25, 'Rain', 3),

-- Very late
(1003, 3, 3, 3,
 '2025-11-21 12:00:00', '2025-11-21 12:35:00', '2025-11-21 13:05:00',
 7.0, 30, 'Snow', 2),

-- On-time / slightly early
(1004, 4, 1, 1,
 '2025-11-21 13:15:00', '2025-11-21 13:45:00', '2025-11-21 13:44:00',
 2.0, 18, 'Clear', 4),

-- A bit late but customer still happy
(1005, 2, 3, 2,
 '2025-11-22 20:00:00', '2025-11-22 20:35:00', '2025-11-22 20:42:00',
 4.2, 22, 'Heavy Traffic', 4);
"""

conn.executescript(orders_sql)
print("Replaced sample orders.")

Replaced sample orders.


In [69]:
run_sql("SELECT * FROM customers;")


,customer_id,customer_name,customer_city
0,1,Alice Johnson,Newark
1,2,Oscar Fernandez,Jersey City
2,3,Maria Lopez,Hoboken
3,4,John Smith,New Brunswick


In [70]:
run_sql("SELECT * FROM orders;")


,order_id,customer_id,restaurant_id,driver_id,order_time,estimated_delivery_time,actual_delivery_time,delivery_distance,preparation_time,weather_conditions,customer_rating
0,1001,1,1,1,2025-11-20 18:00:00,2025-11-20 18:30:00,2025-11-20 18:28:00,3.5,20,Clear,5
1,1002,2,2,2,2025-11-20 19:10:00,2025-11-20 19:40:00,2025-11-20 19:50:00,5.0,25,Rain,3
2,1003,3,3,3,2025-11-21 12:00:00,2025-11-21 12:35:00,2025-11-21 13:05:00,7.0,30,Snow,2
3,1004,4,1,1,2025-11-21 13:15:00,2025-11-21 13:45:00,2025-11-21 13:44:00,2.0,18,Clear,4
4,1005,2,3,2,2025-11-22 20:00:00,2025-11-22 20:35:00,2025-11-22 20:42:00,4.2,22,Heavy Traffic,4


In [71]:
orders_df = run_sql("SELECT * FROM orders;")
orders_df

,order_id,customer_id,restaurant_id,driver_id,order_time,estimated_delivery_time,actual_delivery_time,delivery_distance,preparation_time,weather_conditions,customer_rating
0,1001,1,1,1,2025-11-20 18:00:00,2025-11-20 18:30:00,2025-11-20 18:28:00,3.5,20,Clear,5
1,1002,2,2,2,2025-11-20 19:10:00,2025-11-20 19:40:00,2025-11-20 19:50:00,5.0,25,Rain,3
2,1003,3,3,3,2025-11-21 12:00:00,2025-11-21 12:35:00,2025-11-21 13:05:00,7.0,30,Snow,2
3,1004,4,1,1,2025-11-21 13:15:00,2025-11-21 13:45:00,2025-11-21 13:44:00,2.0,18,Clear,4
4,1005,2,3,2,2025-11-22 20:00:00,2025-11-22 20:35:00,2025-11-22 20:42:00,4.2,22,Heavy Traffic,4


In [72]:
orders_df['order_time'] = pd.to_datetime(orders_df['order_time'])
orders_df['estimated_delivery_time'] = pd.to_datetime(orders_df['estimated_delivery_time'])
orders_df['actual_delivery_time'] = pd.to_datetime(orders_df['actual_delivery_time'])

# delay in minutes
orders_df['delay_minutes'] = (
    orders_df['actual_delivery_time'] - orders_df['estimated_delivery_time']
).dt.total_seconds() / 60

# late = 1 if delay > 0 minutes, else 0
orders_df['late'] = (orders_df['delay_minutes'] > 0).astype(int)

orders_df

,order_id,customer_id,restaurant_id,driver_id,order_time,estimated_delivery_time,actual_delivery_time,delivery_distance,preparation_time,weather_conditions,customer_rating,delay_minutes,late
0,1001,1,1,1,2025-11-20 18:00:00,2025-11-20 18:30:00,2025-11-20 18:28:00,3.5,20,Clear,5,-2.0,0
1,1002,2,2,2,2025-11-20 19:10:00,2025-11-20 19:40:00,2025-11-20 19:50:00,5.0,25,Rain,3,10.0,1
2,1003,3,3,3,2025-11-21 12:00:00,2025-11-21 12:35:00,2025-11-21 13:05:00,7.0,30,Snow,2,30.0,1
3,1004,4,1,1,2025-11-21 13:15:00,2025-11-21 13:45:00,2025-11-21 13:44:00,2.0,18,Clear,4,-1.0,0
4,1005,2,3,2,2025-11-22 20:00:00,2025-11-22 20:35:00,2025-11-22 20:42:00,4.2,22,Heavy Traffic,4,7.0,1


In [73]:
orders_df.to_sql("orders_with_labels", conn, if_exists="replace", index=False)
print("Saved labeled orders to table 'orders_with_labels'.")

Saved labeled orders to table 'orders_with_labels'.


In [74]:
run_sql("SELECT order_id, delay_minutes, late FROM orders_with_labels;")

,order_id,delay_minutes,late
0,1001,-2.0,0
1,1002,10.0,1
2,1003,30.0,1
3,1004,-1.0,0
4,1005,7.0,1


In [75]:
run_sql("""
SELECT late, COUNT(*) AS num_orders
FROM orders_with_labels
GROUP BY late;
""")

,late,num_orders
0,0,2
1,1,3


In [76]:
run_sql("""
SELECT AVG(delay_minutes) AS avg_delay_minutes
FROM orders_with_labels;
""")

,avg_delay_minutes
0,8.8


## 3. Real-World Kaggle Dataset Integration

In this section we load a public Kaggle dataset on food delivery times, clean it, create a late/on-time label, and store the cleaned data back into SQLite for further SQL analysis and modeling.

In [77]:
kaggle_filename = "Food_Delivery_Times.csv"

kaggle_df = pd.read_csv(kaggle_filename)
kaggle_df.head()

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [78]:
kaggle_df.columns

Index(['Order_ID', 'Distance_km', 'Weather', 'Traffic_Level', 'Time_of_Day',
       'Vehicle_Type', 'Preparation_Time_min', 'Courier_Experience_yrs',
       'Delivery_Time_min'],
      dtype='object')

In [79]:
kaggle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order_ID                1000 non-null   int64  
 1   Distance_km             1000 non-null   float64
 2   Weather                 970 non-null    object 
 3   Traffic_Level           970 non-null    object 
 4   Time_of_Day             970 non-null    object 
 5   Vehicle_Type            1000 non-null   object 
 6   Preparation_Time_min    1000 non-null   int64  
 7   Courier_Experience_yrs  970 non-null    float64
 8   Delivery_Time_min       1000 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 70.4+ KB


In [80]:
# this saves kaggle dataset into SQLite as a new table
kaggle_df.to_sql("kaggle_orders_raw", conn, if_exists="replace", index=False)
print("Saved Kaggle dataset to table 'kaggle_orders_raw'.")

Saved Kaggle dataset to table 'kaggle_orders_raw'.


In [81]:
run_sql("SELECT * FROM kaggle_orders_raw LIMIT 5;")

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [82]:
kaggle_clean = kaggle_df.copy()

# drop rows with any missing values
kaggle_clean = kaggle_clean.dropna()

# standardize categorical text columns
categorical_cols = ["Weather", "Traffic_Level", "Time_of_Day", "Vehicle_Type"]
for col in categorical_cols:
    kaggle_clean[col] = kaggle_clean[col].astype(str).str.strip().str.lower()

kaggle_clean.head()

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,windy,low,afternoon,scooter,12,1.0,43
1,738,16.42,clear,medium,evening,bike,20,2.0,84
2,741,9.52,foggy,low,night,scooter,28,1.0,59
3,661,7.44,rainy,medium,afternoon,scooter,5,1.0,37
4,412,19.03,clear,low,morning,bike,16,5.0,68


In [83]:
# late = 1 if delivery time > 30 minutes, else 0
kaggle_clean["late"] = (kaggle_clean["Delivery_Time_min"] > 30).astype(int)

# how many late vs on-time
kaggle_clean["late"].value_counts()

late
1    780
0    103
Name: count, dtype: int64

In [84]:
kaggle_clean.to_sql("kaggle_clean_orders", conn, if_exists="replace", index=False)
print("Saved cleaned Kaggle data to SQL table 'kaggle_clean_orders'.")

Saved cleaned Kaggle data to SQL table 'kaggle_clean_orders'.


In [85]:
run_sql("SELECT * FROM kaggle_clean_orders LIMIT 5;")

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min,late
0,522,7.93,windy,low,afternoon,scooter,12,1.0,43,1
1,738,16.42,clear,medium,evening,bike,20,2.0,84,1
2,741,9.52,foggy,low,night,scooter,28,1.0,59,1
3,661,7.44,rainy,medium,afternoon,scooter,5,1.0,37,1
4,412,19.03,clear,low,morning,bike,16,5.0,68,1


In [86]:
kaggle_clean.describe(include="all")

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min,late
count,883.000000,883.000000,883,883,883,883,883.000000,883.000000,883.000000,883.000000
unique,NaN,NaN,5,3,4,3,NaN,NaN,NaN,NaN
top,NaN,NaN,clear,medium,morning,bike,NaN,NaN,NaN,NaN
freq,NaN,NaN,425,355,281,451,NaN,NaN,NaN,NaN
mean,508.109853,10.051586,NaN,NaN,NaN,NaN,17.019253,4.639864,56.425821,0.883352
std,287.925161,5.688582,NaN,NaN,NaN,NaN,7.260201,2.922172,21.568482,0.321182
min,1.000000,0.590000,NaN,NaN,NaN,NaN,5.000000,0.000000,8.000000,0.000000
25%,263.500000,5.130000,NaN,NaN,NaN,NaN,11.000000,2.000000,41.000000,1.000000
50%,512.000000,10.280000,NaN,NaN,NaN,NaN,17.000000,5.000000,55.000000,1.000000
75%,758.500000,15.025000,NaN,NaN,NaN,NaN,23.000000,7.000000,71.000000,1.000000


## Project Overview

Our project analyzes delivery times for food delivery platforms and predicts whether an order will arrive late or on time. We also study how delivery delays affect customer satisfaction (customer ratings). This connects to data management in data science because the project involves database design, data cleaning, integration, SQL querying, and machine learning.

## Data Sources

We currently use two data sources:

- **Prototype synthetic dataset**: a small hand-crafted set of orders that we use to prototype the database schema, feature engineering steps, and basic SQL analysis.
- **Real-world Kaggle dataset**: *"Food Delivery Time Prediction"* by denkuznetz on Kaggle. This public dataset contains 1,000 real delivery records with features such as distance, weather, traffic level, preparation time, courier experience, and total delivery time in minutes.

The Kaggle dataset is downloaded as a CSV file, uploaded into the Codebench Jupyter environment, and loaded into Python with pandas. We then store both the raw and cleaned versions inside our SQLite database so that we can query them with SQL.

## Progress So Far

### Database Design and Prototype Pipeline

We created a SQLite database (`delivery.db`) and defined four main tables: `customers`, `restaurants`, `drivers`, and `orders`. We populated these tables with example delivery records that include order times, estimated and actual delivery times, distance, preparation time, weather conditions, and customer ratings.

Using Python and pandas, we loaded the `orders` table from SQLite, converted all time fields to proper datetime formats, and computed a new feature `delay_minutes` (actual delivery time minus estimated delivery time). From this, we created a binary label `late` that is 1 if an order arrived after the estimated time and 0 otherwise, and stored this labeled dataset back into the database as a new table `orders_with_labels`.

We also ran several SQL analysis queries on `orders_with_labels` to demonstrate early insights: counting how many orders are late vs on time, computing the average delay in minutes, and comparing average customer ratings between late and on-time deliveries. These queries show how the database and Python processing work together and prepare the data for the machine-learning model.

### Real-World Kaggle Dataset Integration

We loaded the Kaggle dataset into a DataFrame (`kaggle_df`), inspected the schema, and saved it into SQLite as the table `kaggle_orders_raw`. We then created a cleaned version (`kaggle_clean`) by dropping rows with missing values and standardizing categorical columns such as weather, traffic level, time of day, and vehicle type (e.g., converting them to lower case and trimming whitespace).

To obtain a classification target, we defined a binary late/on-time label directly from the real delivery time: we set `late = 1` for deliveries taking more than 30 minutes and `late = 0` otherwise. This reflects a common expectation for food delivery services. We stored this cleaned, labeled dataset back into SQLite as `kaggle_clean_orders` and verified it using SQL queries such as `SELECT * FROM kaggle_clean_orders LIMIT 5;`. We also generated descriptive statistics to understand the distribution of features and the balance between late and on-time deliveries.

## Next Steps

For the final project, we plan to:

- Expand our SQL analysis on the Kaggle data, for example by examining how distance, weather, traffic level, and preparation time affect the probability of a late delivery.
- Engineer additional features such as one-hot encodings for categorical variables, bucketized distance ranges, and indicators for peak vs off-peak times of day.
- Train a logistic regression model that predicts the probability that an order will be late using the features from `kaggle_clean_orders`.
- Split the dataset into training and test sets and evaluate the model using accuracy, precision, recall, and a confusion matrix.
- Interpret the logistic regression coefficients to identify which features are most strongly associated with late deliveries and lower customer satisfaction, and discuss limitations and possible improvements.

Overall, we have completed the database setup, prototype pipeline, integration of a real-world dataset, and initial feature engineering. The next phase will focus on advanced feature design, model training, and evaluation.
